# K-means에 따른 맥주추천 (Train, Test를 이용한 검증)

1. 사용자가 호감을 표시한 맥주의 정보를 가져온다.
2. 호감 맥주의 유사도를 계산한다. (0,1로 나눈 각각의 범주에서 더 많은 숫자를 채택)
3. 호감맥주 유사도를 기준으로 가장 비슷한 맥주를 추천한다.

In [302]:
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

# 데이터 불러오기

In [303]:
train = pd.read_csv("../data/beer4REC_train.csv",encoding='utf-8')
test = pd.read_csv("../data/beer4REC_test.csv",encoding='utf-8')

# 사용자 맥주성향 알아보기

In [304]:
import numpy as np
import pandas as pd

In [305]:
test = train.iloc[1130:,:937]
usr = train.iloc[1130:,:936]
test

,도수(상),도수(중),도수(하),정제수,보리맥아,귀리맥아,설탕,스페인감초분말,호프,효모,...,헤페바이스 둔켈,헤페바이젠,헤페바이젠 + 도펠 복,헬레스 라거,호피 세션 세종,호피 필스너,화이트 에일,가격(상),가격(하),상품명
1130,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,더 미스트레스 세종
1131,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,로덴바흐 그랑 크루
1132,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,아벤티누스 아이스복
1133,0,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,바바리아 프리미엄


# 회원의 선호데이터를 연산을 통하여 얻는다

In [342]:
usr_info=[]
for i in range(len(usr.columns)):
    usr_info.append(usr.iloc[:,i].sum()/len(usr.iloc[:,i]))

In [343]:
usr_info = np.reshape(usr_info,(1,936))

In [344]:
columns = usr.columns

In [345]:
usr_info = pd.DataFrame(data=usr_info,columns=columns)

In [346]:
usr_info.iloc[:,:70]

,도수(상),도수(중),도수(하),정제수,보리맥아,귀리맥아,설탕,스페인감초분말,호프,효모,...,카라멜색소,보리,홉추출물,유자피,체리,라즈베리,빌베리,엘더베리,스트로베리,허브
0,0.25,0.75,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# train데이터를 학습한다.

In [311]:
km = KMeans(n_clusters=168, n_jobs=50, random_state=50)
model=km.fit(train.iloc[:,:936])

In [312]:
labels = model.labels_

# 회원의 선호맥주 군집단을 추출한다

In [317]:
model=km.predict(usr_info)

In [318]:
model

array([3])

# 회원의 군집단을 바탕으로 상품을 추천한다

In [340]:
추천상품=[]
for i in range(len(labels)):
    if (labels[i] == model):
        추천상품.append(train['상품명'][i])

In [341]:
test['상품명']

1130    더 미스트레스 세종
1131    로덴바흐 그랑 크루
1132    아벤티누스 아이스복
1133     바바리아 프리미엄
Name: 상품명, dtype: object

# 회원이 고른 상품목록 제거

In [336]:
for i in range(len(test['상품명'])):
    try:
        추천상품.remove(test['상품명'].iloc[i])
    except:
        pass

# 상품추천 목록

In [339]:
추천상품[:3]

['에치고 필스너 ', '벡스', '베를린 킨들 필스너']